In [ ]:
import pandas as pd
import spacy
import spacy.cli
spacy.cli.download("fr_core_news_sm")

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [ ]:
data = pd.read_csv('source.csv')
data["tokens"]=""

In [ ]:
data.head()

,Connu,tokens
0,Affiche à l'Isep,
1,"Par hasard, en cherchant des sites pour donner...",
2,J'ai connu Teach'r grâce à des amis,
3,J'ai découvert l'application via le play store,
4,J’ai découvert Teach’r sur un site internet,


In [ ]:
wordList=["découvrir", "application", "connaître","teach'", 'r', "trouver", "chercher", "recherche", "contacter", "grâce"]

In [ ]:
nlp = spacy.load('fr_core_news_sm')

In [ ]:
baseDictionary={
    "application store" : ["store", "stor", "play", "moteur", "rechercher"], 
    "isep" : ["isep"],
    "recherche google" : ["internet", "google", "hasard"],
    "ami":["ami", "copain", "copine", "connaissance","bouche","oreille", "pote", 'proche', "collègue", "camarade"],
    "école / université":["école", "université", "fac", "bde", "via","annonce"],
    "radio":["radio","bfm"],
    "leboncoin":["boncoin", "lbc", "bon coin"],
    "publicité":["publicité", "pub"],
    "instagram":["instagram"],
    "facebook":["facebook", "groupe promo", "groupe étudiant", "fb"],
    "réponse par vocal":["https","cdn"],
    "Fondateurs":["eytan", "fondateur", "propriétaire", "créateur", "eliott","alexis" ],
    "gens de confiance":["confiance"],
    "famille":["soeur", "frère","mère","père","parent", "cousin", "cousine", "maman", "papa"],
    "superprof":["superprof", "super prof"],
    "media chinois":["chinois"]
}

def sentenceToToken(sentence):
  if isinstance(sentence, str):
    tokens=[]
    doc=nlp(sentence)
    for token in doc:
        if (not token.is_stop and not token.is_punct) and (token.lemma_ not in wordList):
          tokens.append(token.lemma_)
    return " ".join(tokens)
  return ""

def reduceCommonCase(tokens):
  tokens = tokens.lower()
  for key in baseDictionary:
    for val in baseDictionary[key]:
      if val in tokens:
        return key
  return tokens

data["tokens"]=data["Connu"].apply(lambda x: sentenceToToken(x))
data["tokens"]=data["tokens"].apply(lambda x: reduceCommonCase(x))

In [ ]:
agg_data = data.groupby(["tokens"]).count().sort_values("Connu", ascending=False)

In [ ]:
agg_data_large=agg_data.loc[agg_data["Connu"]>1]

,Connu
tokens,
application store,443
école / université,286
ami,282
recherche google,187
isep,57
facebook,43
leboncoin,19
famille,17
publicité,16


In [ ]:
agg_data_large.to_csv("cleanfile.csv", encoding='utf-8-sig')